To do (to be deleted later):
Week 1 (Nov, 10 -14)
Project Setup & Data Exploration
Objectives:
- Form groups of two students and select a project topic.
- Obtain dataset from Kaggle, UCI ML Repository, or other open sources.
- Understand the problem statement and define clear objectives.
- Perform Exploratory Data Analysis (EDA): check missing values, outliers, and data
imbalance.
- Visualize data using plots (histograms, scatter plots, correlation heatmaps, etc.).
- Identify potential features and target variable.
Deliverables by end of Week 1(Nov. 14):
• Confirmed project topic and dataset (submitted to instructor).
• EDA report (Jupyter Notebook or PDF).
Exploratory Data Analysis Report — it’s a structured summary of your initial
exploration of a dataset before applying any machine learning algorithms.
Week 2 – Nov. 17 - 21
Data Preprocessing & Model Development
Objectives:
- Clean and preprocess dataset (handle missing data, encode categorical variables,
normalize features).
- Split data into training and testing sets.
- Implement baseline models (e.g., Linear Regression, Logistic Regression, KNN).
- Begin experimenting with hyperparameters and model variations.


In [1]:
import pandas as pd

dt = pd.read_csv('CarPrice_Assignment.csv')
print(dt.head())

   car_ID  symboling                   CarName fueltype aspiration doornumber  \
0       1          3        alfa-romero giulia      gas        std        two   
1       2          3       alfa-romero stelvio      gas        std        two   
2       3          1  alfa-romero Quadrifoglio      gas        std        two   
3       4          2               audi 100 ls      gas        std       four   
4       5          2                audi 100ls      gas        std       four   

       carbody drivewheel enginelocation  wheelbase  ...  enginesize  \
0  convertible        rwd          front       88.6  ...         130   
1  convertible        rwd          front       88.6  ...         130   
2    hatchback        rwd          front       94.5  ...         152   
3        sedan        fwd          front       99.8  ...         109   
4        sedan        4wd          front       99.4  ...         136   

   fuelsystem  boreratio  stroke compressionratio horsepower  peakrpm citympg  \